## 3 - Medium Level Example - 1-Axis tracker by hour (gendaylit)

Example demonstrating the use of doing hourly smiulations with Radiance gendaylit for 1-axis tracking. This is a medium level example because it also explores a couple subtopics:

#### Subtopics:
<ul>
    <li> The structure of the tracker dictionary "trackerDict". </li>
    <li> How to calculate GCR </li>
    <li> How to make a cell-level module </li>
    <li> Various methods to use the trackerdictionary for analysis. </li>
</ul>
 
#### Doing full year simulations with gendaylit: 

Performing the simulation hour by hour requires either a good computer or some patience, since there are ~4000 daylight-hours in the year. With a 32GB RAM, Windows 10 i7-8700 CPU @ 3.2GHz with 6 cores this takes 1 day. The code also allows for multiple cores or HPC use -- there is documentation/examples inside the software at the moment, but that is an advanced topic. The procedure can be broken into shorter steps for one day or a single timestamp simulation which is exemplified below.

### Steps:
<ol>
    <li> <a href='#step1'> Load bifacial_radiance </a></li> 
    <li> <a href='#step2'> Define all your system variables </a></li> 
    <li> <a href='#step3'> Create Radiance Object, Set Albedo and Weather </a></li> 
    <li> <a href='#step4'> Make Module: Cell Level Module Example </a></li>    
    <li> <a href='#step5'> Calculate GCR</a></li> 
    <li> <a href='#step6'> Set Tracking Angles </a></li> 
    <li> <a href='#step7'> Generate the Sky </a></li> 
    <li> <a href='#step8'> Make Scene 1axis </a></li> 
    <li> <ol type="A"><li><a href='#step9a'> Make Oct and AnalyzE 1 HOUR </a></li> 
    <li> <a href='#step9b'> Make Oct and Analye Range of Hours </a></li> 
        <li> <a href='#step9c'>  Make Oct and Analyze All Tracking Dictionary </a></li> </ol>
</ol>

And finally:  <ul> <a href='#condensed'> Condensed Version: All Tracking Dictionary </a></ul>   

<a id='step1'></a>


## 1. Load bifacial_radiance 

#### Pay attention: different importing method:

So far we've used "from bifacial_radiance import *" to import all the bifacial_radiance files into our working space in jupyter. For this journal we will do a "import bifacial_radiance" . This method of importing requires a different call for some functions as you'll see below. For example, instead of calling demo = RadianceObj(path = testfolder) as on Tutorial 2, in this case we will neeed to do demo = bifacial_radiance.RadianceObj(path = testfolder). 

In [1]:
import bifacial_radiance
import numpy as np
import os # this operative system to do teh relative-path testfolder for this example.
import pprint    # We will be pretty-printing the trackerdictionary throughout to show its structure.
from pathlib import Path

<a id='step2'></a>

## 2. Define all your system variables

Just like in the condensed version show at the end of Tutorial 2, for this tutorial we will be starting all of our system variables from the beginning of the jupyter journal, instead than throughout the different cells (for the most part)

In [3]:
testfolder = Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP'

simulationName = 'Tutorial 3'    # For adding a simulation name when defning RadianceObj. This is optional.
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = True    # the scene will rotate around the torque tube, and not the bottom edge of the module
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)

# starting in v0.4.0, some torque tube parameters are passed separately as a dictionary.
tubeParams = {'diameter':diameter,
              'tubetype':tubetype,
              'material':material,
              'axisofrotation':axisofrotationTorqueTube}


<a id='step3'></a>

## 3. Create Radiance Object, Set Albedo and Weather

Same steps as previous two tutorials, so condensing it into one step. You hopefully have this down by now! :)


<div class="alert alert-warning">
Notice that we are doing bifacial_radiance.RadianceObj because we change the import method for this example!
</div>

We now constrain the days of our analysis in the readWeatherFile import step.  For this example we are doing just two days in January. Format has to be a 'MM_DD' or 'YYYY-MM-DD_HHMM'

In [4]:
demo = bifacial_radiance.RadianceObj(simulationName, path = str(testfolder))  # Adding a simulation name. This is optional.
demo.setGround(albedo) 
epwfile = demo.getEPW(lat=lat, lon=lon) 

starttime = '01_13';  endtime = '01_14'
metdata = demo.readWeatherFile(weatherFile=epwfile, starttime=starttime, endtime=endtime) 


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Loading albedo, 1 value(s), 0.213 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly  WeatherFile for the year for purposes of saving Gencumulativesky temporal weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


<a id='step4'></a>

## 4. Make Module: Cell Level Module Example

Instead of doing a opaque, flat single-surface module, in this tutorial we will create a module made up by cells. We can define variuos parameters to make a cell-level module, such as cell size and spacing between cells. To do this, we will pass a dicitonary with the needed parameters to makeModule, as shown below.  

NOTE: in v0.4.0 some keywords were changed in makeModule - specifically the cellModule and tubeParams dictionary inputs.

<div class="alert alert-warning">
Since we are passing a cell-level dictionary, the values for module's x and y of the module will be calculated by the software -- no need to pass them (and if you do, they'll just get ignored)
    </div>

In [5]:
numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

cellModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
              'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=moduletype, torquetube=True,  
                xgap=xgap, ygap=ygap, zgap=zgap, numpanels=numpanels, 
                cellModule=cellModuleParams, 
                tubeParams=tubeParams)


Module Name: Custom_Cell-Level_Module
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Custom_Cell-Level_Module updated in module.json


<a id='step5'></a>

## 5. Calculate GCR

In this example we passed the parameter "pitch". Pitch is the spacing between rows (for example, between hub-posts) in a field.
To calculate Ground Coverage Ratio (GCR), we must relate the pitch to the collector-width by:
    
![GCR = CW / pitch](../images_wiki/Journal3Pics/Equation_GCR.PNG)

The collector width for our system must consider the number of panels and the y-gap:
    
![CW](../images_wiki/Journal3Pics/Equation_CW.PNG)
    
Collector Width gets saved in your module parameters (and later on your scene and trackerdict) as "sceney". You can calculate your collector width with the equation, or you can use this method to know your GCR:

In [6]:
# For more options on makemodule, see the help description of the function.  
# Details about the module are stored in the new ModuleObj 
CW = mymodule.sceney
gcr = CW / pitch
print ("The GCR is :", gcr)
print(f"ModuleObj data keys: {mymodule.keys}")

The GCR is : 0.4284
ModuleObj data keys: dict_keys(['x', 'y', 'z', 'modulematerial', 'scenex', 'sceney', 'scenez', 'numpanels', 'bifi', 'text', 'modulefile', 'offsetfromaxis', 'xgap', 'ygap', 'zgap', 'cellModule', 'torquetube'])


<a id='step6'></a>

## 6. Set Tracking Angles

This function will read the weather file, and based on the sun position it will calculate the angle the tracker should be at for each hour. It will create metdata files for each of the tracker angles considered.

For doing hourly simulations, remember to set **cumulativesky = False** here!

In [7]:
trackerdict = demo.set1axis(metdata=metdata, limit_angle=limit_angle, backtrack=backtrack, 
                            gcr=gcr, cumulativesky=False)

In [8]:
print ("Trackerdict created by set1axis: %s " % (len(demo.trackerdict))) 


Trackerdict created by set1axis: 20 


set1axis initializes the trackerdictionary Trackerdict. Trackerdict contains all hours selected from the weatherfile as keys. For example: trackerdict['2021-01-13_1200']. It is a return variable on many of the 1axis functions, but it is also stored inside of your Radiance Obj (i.e. demo.trackerdict). In this journal we are storing it as a variable to mute the option (otherwise it prints the returned trackerdict contents every time)


In [9]:
pprint.pprint(trackerdict['2021-01-13_1200'])


{'dhi': 200, 'ghi': 249, 'surf_azm': 90.0, 'surf_tilt': 21.2, 'theta': -21.2}


All of the following functions add up elements to trackerdictionary to keep track (ba-dum tupzz) of the Scene and simulation parameters. In advanced journals we will explore the inner structure of trackerdict. For now, just now it exists :)

<a id='step7'></a>

## 7. Generate the Sky


We will create skies for each hour we want to model with the function gendaylit1axis. 

For this example we are doing just two days in January. The ability to limit the time using gendaylit1axis is deprecated.  Use readWeatherFile instead.

In [11]:

trackerdict = demo.gendaylit1axis() 

Creating ~20 skyfiles.  Takes 1-2 minutes
Created 19 skyfiles in /skies/


Since we passed startdate and enddate to gendaylit, it will prune our trackerdict to only the desired days.
Let's explore our trackerdict:

In [12]:
trackerkeys = sorted(trackerdict.keys())
print ("Trackerdict option of hours are: ", trackerkeys)
print ("")
print ("Contents of trackerdict for sample hour:")
pprint.pprint(trackerdict[trackerkeys[0]])

Trackerdict option of hours are:  ['2021-01-13_0800', '2021-01-13_0900', '2021-01-13_1000', '2021-01-13_1100', '2021-01-13_1200', '2021-01-13_1300', '2021-01-13_1400', '2021-01-13_1500', '2021-01-13_1600', '2021-01-13_1700', '2021-01-14_0800', '2021-01-14_0900', '2021-01-14_1000', '2021-01-14_1100', '2021-01-14_1200', '2021-01-14_1300', '2021-01-14_1400', '2021-01-14_1500', '2021-01-14_1600']

Contents of trackerdict for sample hour:
{'dhi': 22,
 'ghi': 23,
 'skyfile': 'skies\\sky2_37.5_-77.33_2021-01-13_0800.rad',
 'surf_azm': 90.0,
 'surf_tilt': 3.74,
 'theta': -3.74}


<a id='step8'></a>

## 8. Make Scene 1axis

We can use gcr or pitch fo our scene dictionary.

In [13]:
# making the different scenes for the 1-axis tracking for the dates in trackerdict2.

sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows}  

trackerdict = demo.makeScene1axis(trackerdict=trackerdict, module=mymodule, sceneDict=sceneDict) 


Making ~19 .rad files for gendaylit 1-axis workflow (this takes a minute..)
19 Radfiles created in /objects/


The scene parameteres are now stored in the trackerdict. To view them and to access them:
    

In [14]:
pprint.pprint(trackerdict[trackerkeys[0]])

{'clearance_height': 2.160279548406131,
 'dhi': 22,
 'ghi': 23,
 'radfile': 'objects\\1axis2021-01-13_0800__C_2.16028_rtr_10.00000_tilt_3.74000_20modsx7rows_origin0,0.rad',
 'scene': <bifacial_radiance.main.SceneObj object at 0x00000285696E3280>,
 'skyfile': 'skies\\sky2_37.5_-77.33_2021-01-13_0800.rad',
 'surf_azm': 90.0,
 'surf_tilt': 3.74,
 'theta': -3.74}


In [15]:
pprint.pprint(demo.trackerdict[trackerkeys[5]]['scene'].__dict__)

{'gcr': 0.4284,
 'module': <bifacial_radiance.main.ModuleObj object at 0x000002856785F9D0>,
 'moduleDict': {'bifi': 1,
                'cellModule': {'numcellsx': 6,
                               'numcellsy': 12,
                               'xcell': 0.156,
                               'xcellgap': 0.02,
                               'ycell': 0.156,
                               'ycellgap': 0.02},
                'modulefile': 'objects\\Custom_Cell-Level_Module.rad',
                'modulematerial': 'black',
                'numpanels': 2,
                'offsetfromaxis': 0,
                'scenex': 1.046,
                'sceney': 4.284,
                'scenez': 0.1,
                'text': '! genbox black cellPVmodule 0.156 0.156 0.02 | xform '
                        '-t -0.44 -2.142 0 -a 6 -t 0.176 0 0 -a 12 -t 0 0.176 '
                        '0 -a 2 -t 0 2.192 0\r\n'
                        '! genbox black octtube1a 1.046 0.04142135623730951 '
                        '

<a id='step9a'></a>

## 9. Make Oct and Analyze 

### A. Make Oct and Analyze 1Hour

There are various options now to analyze the trackerdict hours we have defined. We will start by doing just one hour, because it's the fastest. Make sure to select an hour that exists in your trackerdict!

Options of hours:

In [16]:
pprint.pprint(trackerkeys)

['2021-01-13_0800',
 '2021-01-13_0900',
 '2021-01-13_1000',
 '2021-01-13_1100',
 '2021-01-13_1200',
 '2021-01-13_1300',
 '2021-01-13_1400',
 '2021-01-13_1500',
 '2021-01-13_1600',
 '2021-01-13_1700',
 '2021-01-14_0800',
 '2021-01-14_0900',
 '2021-01-14_1000',
 '2021-01-14_1100',
 '2021-01-14_1200',
 '2021-01-14_1300',
 '2021-01-14_1400',
 '2021-01-14_1500',
 '2021-01-14_1600']


In [17]:
demo.makeOct1axis(singleindex='2021-01-13_0800')
results = demo.analysis1axis(singleindex='2021-01-13_0800')
print('\n\nHourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))


Making 1 octfiles in root directory.
Created 1axis_2021-01-13_0800.oct
Linescan in process: 1axis_2021-01-13_0800_Front
Linescan in process: 1axis_2021-01-13_0800_Back
Saved: results\irr_1axis_2021-01-13_0800.csv
Index: 2021-01-13_0800. Wm2Front: 21.997914555555557. Wm2Back: 2.7988606666666667


Hourly bifi gain: 0.127


The trackerdict now contains information about the octfile, as well as the Analysis Object results

In [18]:
print ("\n Contents of trackerdict for sample hour after analysis1axis: ")
pprint.pprint(trackerdict[trackerkeys[0]])



 Contents of trackerdict for sample hour after analysis1axis: 
{'AnalysisObj': <bifacial_radiance.main.AnalysisObj object at 0x00000285696E3B80>,
 'Wm2Back': [2.924148666666667,
             2.8673869999999995,
             2.812069333333333,
             2.689787666666667,
             2.990579,
             2.6426313333333336,
             2.727484666666667,
             2.7545403333333334,
             2.7811179999999998],
 'Wm2Front': [23.61272,
              23.61375333333333,
              23.61418,
              23.614606666666663,
              8.734767666666665,
              23.692423333333334,
              23.696346666666667,
              23.699593333333336,
              23.70284],
 'backRatio': [0.12383261369520206,
               0.12142354228839426,
               0.11907888626439997,
               0.11389873250486052,
               0.34233728667158164,
               0.11153438218509862,
               0.11509662685161376,
               0.11622242087328895,
      

In [19]:
pprint.pprint(trackerdict[trackerkeys[0]]['AnalysisObj'].__dict__)

{'Wm2Back': [2.924148666666667,
             2.8673869999999995,
             2.812069333333333,
             2.689787666666667,
             2.990579,
             2.6426313333333336,
             2.727484666666667,
             2.7545403333333334,
             2.7811179999999998],
 'Wm2Front': [23.61272,
              23.61375333333333,
              23.61418,
              23.614606666666663,
              8.734767666666665,
              23.692423333333334,
              23.696346666666667,
              23.699593333333336,
              23.70284],
 'backRatio': [0.12383261369520206,
               0.12142354228839426,
               0.11907888626439997,
               0.11389873250486052,
               0.34233728667158164,
               0.11153438218509862,
               0.11509662685161376,
               0.11622242087328895,
               0.1173277409904893],
 'mattype': ['a9.3.a2.2.0.cellPVmodule.6457',
             'a9.3.a2.4.0.cellPVmodule.6457',
             'a9.3.a2.7.0

<a id='step9b'></a>

### B. Make Oct and Analye Range of Hours

You could do a list of indices following a similar procedure:

In [20]:
for time in ['2021-01-13_0900','2021-01-13_1300']:  
    demo.makeOct1axis(singleindex=time)
    results=demo.analysis1axis(singleindex=time)

print('Accumulated hourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))


Making 1 octfiles in root directory.
Created 1axis_2021-01-13_0900.oct
Linescan in process: 1axis_2021-01-13_0900_Front
Linescan in process: 1axis_2021-01-13_0900_Back
Saved: results\irr_1axis_2021-01-13_0900.csv
Index: 2021-01-13_0900. Wm2Front: 94.37003555555556. Wm2Back: 7.1145364814814815

Making 1 octfiles in root directory.
Created 1axis_2021-01-13_1300.oct
Linescan in process: 1axis_2021-01-13_1300_Front
Linescan in process: 1axis_2021-01-13_1300_Back
Saved: results\irr_1axis_2021-01-13_1300.csv
Index: 2021-01-13_1300. Wm2Front: 218.52505185185183. Wm2Back: 30.70506222222222
Accumulated hourly bifi gain: 0.121


Note that the bifacial gain printed above is for the accumulated irradiance between the hours modeled so far. 
That is, demo.Wm2Back and demo.Wm2Front are for January 13, 8AM, 9AM and  1 PM. Compare demo.Wm2back below with what we had before:

In [21]:
demo.Wm2Back

array([37.49391533, 36.02065467, 34.77338167, 32.50565867, 84.072514  ,
       33.24010967, 34.53906433, 35.82348067, 37.09735533])

To print the specific bifacial gain for a specific hour, you can use the following:

In [22]:
sum(trackerdict['2021-01-13_1300']['AnalysisObj'].Wm2Back) / sum(trackerdict['2021-01-13_1300']['AnalysisObj'].Wm2Front)

0.14051049049991116

<a id='step9c'></a>

### C. Make Oct and Analyze All Tracking Dictionary

This takes considerably more time, depending on the number of entries on the trackerdictionary. If no **starttime** and **endtime** were specified on STEP **readWeatherFile, this will run ALL of the hours in the year (~4000 hours).**


In [ ]:
demo.makeOct1axis()
results = demo.analysis1axis()
print('Accumulated hourly bifi gain for all the trackerdict: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))



Making 19 octfiles in root directory.
Created 1axis_2021-01-13_0800.oct
Created 1axis_2021-01-13_0900.oct
Created 1axis_2021-01-13_1000.oct
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct
Created 1axis_2021-01-13_1300.oct
Created 1axis_2021-01-13_1400.oct
Created 1axis_2021-01-13_1500.oct
Created 1axis_2021-01-13_1600.oct
Created 1axis_2021-01-13_1700.oct
Created 1axis_2021-01-14_0800.oct
Created 1axis_2021-01-14_0900.oct
Created 1axis_2021-01-14_1000.oct
Created 1axis_2021-01-14_1100.oct
Created 1axis_2021-01-14_1200.oct
Created 1axis_2021-01-14_1300.oct
Created 1axis_2021-01-14_1400.oct
Created 1axis_2021-01-14_1500.oct
Created 1axis_2021-01-14_1600.oct
Linescan in process: 1axis_2021-01-13_0800_Front
Linescan in process: 1axis_2021-01-13_0800_Back
Saved: results\irr_1axis_2021-01-13_0800.csv
Index: 2021-01-13_0800. Wm2Front: 23.54898814814815. Wm2Back: 2.8068647777777773
Linescan in process: 1axis_2021-01-13_0900_Front
Linescan in process: 1axis_2021-01-13_0900_

<div class="alert alert-warning">
Remember you should clean your results first! This will have torquetube and sky results if performed this way so don't trust this simplistic bifacial_gain examples.
</div>

<a id='condensed'></a>

### Condensed Version: All Tracking Dictionary

This is the summarized version to run gendaylit for each entry in the trackingdictionary.

In [ ]:
import bifacial_radiance
import os 

simulationName = 'Tutorial 3'
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = True  # the scene will rotate around the torque tube, and not the bottom edge of the module
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube material (0% reflectivity)
tubeParams = {'diameter':diameter,
              'tubetype':tubetype,
              'material':material,
              'axisofrotation':axisofrotationTorqueTube}

startdate = '11_06'     
enddate = '11_06'
demo = bifacial_radiance.RadianceObj(simulationName, path=testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readWeatherFile(epwfile, starttime=startdate, endtime=enddate)  
mymodule = bifacial_radiance.ModuleObj(name=moduletype, torquetube=torquetube, xgap=xgap, ygap=ygap,   
                zgap=zgap, numpanels=numpanels,cellModule=cellModuleParams, tubeParams=tubeParams)
sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}  
demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
demo.gendaylit1axis(startdate=startdate, enddate=enddate)
demo.makeScene1axis(module=mymodule,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()
demo.analysis1axis()